In [1]:
import polars as pl
import pandas as pd 
import os
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


account_name = os.getenv('ACCOUNT_NAME')
account_key = os.getenv('ACCOUNT_KEY')
storage_options = {
    "account_name": account_name,
    "account_key":  account_key,
}


In [2]:
need_call = ['2025-11-19','2025-11-20','2025-11-21','2025-11-22']
df = pl.scan_parquet(
    # "az://adjuststbuatprocessed/output/2025-11-21.parquet",
    # "az://adjuststbuatprocessed/live/output/2025-10-07.parquet",
    # "az://adjuststbuatprocessed/live/processing/dt=2025-10-07/*.parquet",
    # [f"az://adjuststbuatprocessed/output/{i}.parquet" for i in need_call],
    [f"az://adjuststbuatprocessed/live/output/{i}.parquet" for i in need_call],
    storage_options=storage_options,
    glob=True,                 # rất quan trọng để expand '*'
)


In [3]:
import re
colnames = [re.findall(r'(?<=\{)[^}]+(?=\})', i)[0] for i in list(df.collect_schema())[:-1]] + [list(df.collect_schema())[-1]]
col_dic = {key:val for key, val in zip(list(df.collect_schema()), colnames)}
df = df.rename(col_dic)
df = df.with_columns(
    (pl.from_epoch('created_at', time_unit='s') + pl.duration(hours = 7)).alias('created_at_convert'),
    (pl.from_epoch('installed_at', time_unit='s') + pl.duration(hours = 7)).alias('installed_at_convert'),
    (pl.from_epoch('engagement_time', time_unit='s') + pl.duration(hours = 7)).alias('engagement_time_convert'),
).with_columns(
    pl.col('created_at_convert').dt.date().alias('created_at_convert_date'),
    pl.col('installed_at_convert').dt.date().alias('installed_at_convert_date'),
    pl.col('engagement_time_convert').dt.date().alias('engagement_time_convert_date'),
)


In [4]:
selected = ['adid','app_version_short','network_name','campaign_name','activity_kind', 'event_name', 'created_at_convert','created_at_convert_date','partner_parameters','publisher_parameters','label', 'first_tracker','tracker_name', 'adgroup_name', 'creative_name']
device_cols = [
    'reftag',
    # 'app_version_short',
    'device_manufacturer',
    'city','country','country_subdivision',
    'device_model','device_name','device_type','environment',
    'os_name','os_version', 'installed_at_convert', 'installed_at_convert_date',
    'idfa','idfv', 'gps_adid'
]
cols = selected + device_cols


In [6]:
adids = df.filter(
    pl.col.event_name.is_in(['ekyc_success','open_account_complete'])
).select(pl.col.adid.unique()).collect(engine = 'streaming').to_pandas().adid.to_list()
print(adids)

['45c2b4a1998a666ef8d657c625038003', '1cb766b09476beb896336fe628063a97', '5d3b930e6f8fdd11e5f36e5554294e59', 'ff7ea224b54b3222dc08a002cb798c15', '8253c2b4a546a0cbd95a4ff9fbc07a68', '31a8ec5e0e27d3d386ef8ce1efd3b53b', 'a7cfd8f64dce1918b2a16ebc374c6110', 'b232762d4b40446cc5c23bd0a6c0dce4', '79e1d333f9e329e95c7deae9489c4d1a', '2138923c14cd2ab958c079e0800de140', 'b149733382a8bf7e0afb5b2905eb1c97', '46f965d5b859cdfe4ff0144b98feab01', 'ed432bed72c055039f032acf4bbb3273', '91cd16d4a85c94e390d4279c43d9ad5d', '5dc84d5cebaaf5c016fe29e155768be9', 'f455a83fb92adf1f26d9a604e44b3a4a', '17eb1961c984eee5d0e14510c2eecee6', '5aa606a5980f970badea03cd1ab0fc22', '9bf661eea388e979baec832a113261f8', '2139139c65efb54b3318235ebdf21438', 'd0f593acf8a8c72c24471327d01832a4', 'f10c50d6af815618120ea5145235deb4', '6ae2a776d34670694bb65979570cf643', 'dfe4e13fc07ebe7ce28344a45ec767e6', '8fea304a0f86598afc4ca4dbdd8aa531', '818702d19278f6a38942952740a39ec9', 'd2fe28c042bd0f596efe1aee604a7468', '83e8bb3a29b1788cabdb4b70cc

In [11]:
from datetime import datetime, timezone, timedelta
timezone_offset = 7.0  # Pacific Standard Time (UTC−08:00)
tzinfo = timezone(timedelta(hours=timezone_offset))
# datetime.now(tzinfo)
now = datetime.now(tz=tzinfo).strftime("%Y%m%dT%H%M%S")
date = datetime.now(tz=tzinfo).strftime("%Y%m%d")
print((now,date))
import os
os.makedirs(f'AdhocOutput/{date}',exist_ok=True)
df.select(cols).filter(pl.col.adid.is_in(adids)).sort(['adid','created_at_convert']).collect().to_pandas().to_csv(f'AdhocOutput/{date}/prod_{'-'.join(need_call)}_{now}.csv', index = False)

('20251124T131441', '20251124')


In [ ]:
# cifs = ['19007449']
# adids = df.filter(
#     pl.col.partner_parameters.str.contains_any(cifs) | pl.col.label.str.contains("0886962308")
# ).select(pl.col.adid.unique()).collect(engine = 'streaming').to_pandas().adid.to_list()
# print(adids)

['d4b21bd40b091f1e2293b6c5c1aedf82']


In [ ]:
date = "20251121"
import os
os.makedirs(f'AdhocOutput/{date}',exist_ok=True)
df.select(cols).filter(pl.col.adid.is_in(adids)).sort(['adid','created_at_convert']).collect().to_pandas().to_csv(f'AdhocOutput/{date}/uat_{'-'.join(cifs)}_{date}T1600.csv', index = False)

In [9]:
adids = df.filter(
    pl.col.created_at_convert.is_between(pl.datetime(2025,11,18,10,41),pl.datetime(2025,11,18,10,43)),
    pl.col.event_name.is_in(["ekyc_success",'open_account_complete'])
          ).select(pl.col.adid.unique()).collect(engine = 'streaming').to_pandas().adid.to_list()
print(adids)

['a4393d2e7ca86ea24f871a470085e3b1', 'd11c8e99782ea1c789c2e685acb0b99b', 'c9b0dedeadd76975e1b4d33ef857c144', '7948a5db70e10e3f1aa2cb3eb0c8c671', '77e5980762c2c17fc673781f0e2412fb', '09d394832cba44b3344c6479fa0bac83', '4becb4009c097b298117c86da9d9caa3']


In [10]:
df.select(cols).filter(pl.col.adid.is_in(adids)).sort(['adid','created_at_convert']).collect().to_pandas().to_csv(f'prod_20251118T1041-43_20251120T1000.csv', index = False)